In [1]:
from math import sqrt
import copy
import  traceback
import shutil

import numpy as np  # linear algebra
import pydicom
import os
import matplotlib.pyplot as plt
import cv2
from pydicom.uid import UID
from PIL import Image

In [2]:
def load_scan(path):
    slices = [pydicom.dcmread(path + '/' + s) for s in filter(lambda x: x.endswith('.dcm'), os.listdir(path))]
    slices.sort(key=lambda x: float(x.InstanceNumber))
    return slices

## 阳性数据

In [97]:
def generate_image(input_folder):
    for patient in os.listdir(input_folder):
        if os.path.isfile(os.path.join(input_folder, patient)):
            continue
        print(f'****Processing {patient}****')
        for scan in os.listdir(os.path.join(input_folder, patient)):
            if scan != '1':
                continue
            name = patient #name = patient.split('-')[0]
            image_path = os.path.join(input_folder, patient, scan, 'images')
            if os.path.exists(image_path):
                shutil.rmtree(image_path)
            os.mkdir(image_path)

            ct = load_scan(os.path.join(input_folder, patient, scan))

            for i in range(len(ct)):
                img = ct[i].pixel_array.astype(np.int16)
                intercept = ct[i].RescaleIntercept
                slope = ct[i].RescaleSlope
                if slope != 1:
                    img = (slope * img.astype(np.float64)).astype(np.int16)
                img += np.int16(intercept)
                img = np.clip(img, -100, 500)
                img = ((img+100)/600*255).astype(np.uint8)
                img = Image.fromarray(img)
                img.save(os.path.join(image_path, f'{name}_{i:04d}.png'))

input_folder = '/nfs3-p1/zsxm/dataset/2021-9-19/'
generate_image(input_folder)

****Processing baozuoshi-J-16-79****
****Processing changzhenxuan-J-15-55****
****Processing chenaijiao-J-26-41****
****Processing chencaiying-J-19-79****
****Processing chenfuliang-S-21-31-J-21-84****
****Processing chenganyao-J-19-40****
****Processing chenhui-S-17-25-J-17-87****
****Processing chenjufa-J-16-66****
****Processing digenlan-J-23-50****
****Processing fangchunfeng-J-16-48****
****Processing fangzhuxiang-J-18-61****
****Processing fudehe-S-20-33-J-20-70****
****Processing ganxiaobin-J-23-51****
****Processing huangwanman-S-15-30****
****Processing kongxiaoqing-S-19-32-J-27-94****
****Processing liangjianjun-S-17-23****
****Processing linfaqiu-S-18-29****
****Processing linjiaxiang-J-21-72****
****Processing linrichun-J-16-72****
****Processing lirucou-J-19-59****
****Processing liubihai-S-23-26-J-23-93****
****Processing liuhongfu-J-57-200****
****Processing liuzhangliu-J-22-28****
****Processing nixueya-J-19-75****
****Processing pancunxiao-S-16-30-J-16-70****
****Proce

In [98]:
def move_together(input_folder, dst_path):
    if not os.path.exists(dst_path):
        os.mkdir(dst_path)

    for patient in os.listdir(input_folder):
        if os.path.isfile(os.path.join(input_folder, patient)):
            continue
        print(f'****Processing {patient}****')
        name = patient #name = patient.split('-')[0]
        if os.path.exists(os.path.join(dst_path, name)):
            print(f"\tremove {os.path.join(dst_path, name)}")
            shutil.rmtree(os.path.join(dst_path, name))

        try:
            shutil.copytree(os.path.join(input_folder, patient, '1', 'images'), os.path.join(dst_path, name))
        except:
            traceback.print_exc()
        
input_folder = '/nfs3-p1/zsxm/dataset/2021-9-19/'
dst_path = '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/new'
move_together(input_folder, dst_path)

****Processing baozuoshi-J-16-79****
****Processing changzhenxuan-J-15-55****
****Processing chenaijiao-J-26-41****
****Processing chencaiying-J-19-79****
****Processing chenfuliang-S-21-31-J-21-84****
****Processing chenganyao-J-19-40****
****Processing chenhui-S-17-25-J-17-87****
****Processing chenjufa-J-16-66****
****Processing digenlan-J-23-50****
****Processing fangchunfeng-J-16-48****
****Processing fangzhuxiang-J-18-61****
****Processing fudehe-S-20-33-J-20-70****
****Processing ganxiaobin-J-23-51****
****Processing huangwanman-S-15-30****
****Processing kongxiaoqing-S-19-32-J-27-94****
****Processing liangjianjun-S-17-23****
****Processing linfaqiu-S-18-29****
****Processing linjiaxiang-J-21-72****
****Processing linrichun-J-16-72****
****Processing lirucou-J-19-59****
****Processing liubihai-S-23-26-J-23-93****
****Processing liuhongfu-J-57-200****
****Processing liuzhangliu-J-22-28****
****Processing nixueya-J-19-75****
****Processing pancunxiao-S-16-30-J-16-70****
****Proce

In [99]:
input_path = '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/new'

for patient in os.listdir(input_path):
    image_path = os.path.join(input_path, patient, 'images')
    if os.path.exists(image_path):
        shutil.rmtree(image_path)
    os.mkdir(image_path)
    for img in os.listdir(os.path.join(input_path, patient)):
        if os.path.isfile(os.path.join(input_path, patient, img)):
            shutil.move(os.path.join(input_path, patient, img), os.path.join(image_path, img))

In [100]:
input_path = '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/pred_test'
output_path = '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/new'

for patient in os.listdir(input_path):
    print(f'Processing {patient}')
    image_path = os.path.join(output_path, patient, 'pred_images')
    if os.path.exists(image_path):
        shutil.rmtree(image_path)
    os.mkdir(image_path)
    for file in os.listdir(os.path.join(input_path, patient)):
        if os.path.isfile(os.path.join(input_path, patient, file)):
            shutil.move(os.path.join(input_path, patient, file), os.path.join(image_path, file))
        elif os.path.isdir(os.path.join(input_path, patient, file)):
            shutil.move(os.path.join(input_path, patient, file), os.path.join(output_path, patient, file))

Processing baozuoshi-J-16-79
Processing changzhenxuan-J-15-55
Processing chenaijiao-J-26-41
Processing chencaiying-J-19-79
Processing chenfuliang-S-21-31-J-21-84
Processing chenganyao-J-19-40
Processing chenhui-S-17-25-J-17-87
Processing chenjufa-J-16-66
Processing digenlan-J-23-50
Processing fangchunfeng-J-16-48
Processing fangzhuxiang-J-18-61
Processing fudehe-S-20-33-J-20-70
Processing ganxiaobin-J-23-51
Processing huangwanman-S-15-30
Processing kongxiaoqing-S-19-32-J-27-94
Processing liangjianjun-S-17-23
Processing linfaqiu-S-18-29
Processing linjiaxiang-J-21-72
Processing linrichun-J-16-72
Processing lirucou-J-19-59
Processing liubihai-S-23-26-J-23-93
Processing liuhongfu-J-57-200
Processing liuzhangliu-J-22-28
Processing nixueya-J-19-75
Processing pancunxiao-S-16-30-J-16-70
Processing shanghongjun-J-22-84
Processing shenjian-J-15-81
Processing siyoulin-S-16-26
Processing sunhongliang-S-23-34
Processing wangjicheng-J-14-22
Processing wangxiaofu-S-19-29-J-19-65
Processing xiafangzh

In [101]:
input_path = '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/new'
error_patient_list = []

def find_coordinate(height, width, label_file, aorta):
    with open(label_file, 'r') as f:
        lines = f.readlines()
    assert len(lines) <= 2, f'label.txt应该存储不多于2个label：{label_file.split("/")[-1]}'
    if len(lines) == 1:
        assert aorta == 'j', f'如果只有一个label那么此时应为降主动脉, 但实际为{aorta}：{label_file.split("/")[-1]}'
        corr = list(map(lambda x: float(x), lines[0].split()))
        x, y, w, h = corr[1], corr[2], corr[3], corr[4]
        assert 0.25 < x < 0.75 and 0.25 < y < 0.75, f'边界框中心({x}, {y})出界：{label_file.split("/")[-1]}'
    else:
        corr1, corr2 = list(map(lambda x: float(x), lines[0].split())), list(map(lambda x: float(x), lines[1].split()))
        assert 0.25 < corr1[1] < 0.75 and 0.25 < corr1[2] < 0.75, f'边界框1中心({corr1[1]}, {corr1[2]})出界：{label_file.split("/")[-1]}'
        assert 0.25 < corr2[1] < 0.75 and 0.25 < corr2[2] < 0.75, f'边界框2中心({corr2[1]}, {corr2[2]})出界：{label_file.split("/")[-1]}'
        if aorta == 's':
            x, y, w, h = (corr1[1], corr1[2], corr1[3], corr1[4]) if corr1[2] < corr2[2] else (corr2[1], corr2[2], corr2[3], corr2[4])
        elif aorta == 'j':
            x, y, w, h = (corr1[1], corr1[2], corr1[3], corr1[4]) if corr1[2] > corr2[2] else (corr2[1], corr2[2], corr2[3], corr2[4])
        else:
            raise Exception(f'aorta 应该为"s"或"j"其中之一: {label_file.split("/")[-1]}')
    w, h = max(w, h), max(w, h)
    return int(width*(x-w/2)), int(height*(y-h/2)), int(width*(x+w/2)+1), int(height*(y+h/2)+1)

for patient in sorted(os.listdir(input_path)):
    print(f'******Processing {patient}******')
    image_path = os.path.join(input_path, patient, 'images')
    label_path = os.path.join(input_path, patient, 'labels')
    crop_path = os.path.join(input_path, patient, 'crops')
    if os.path.exists(crop_path):
        shutil.rmtree(crop_path)
    os.mkdir(crop_path)
    pl = patient.lower().split('-')
    for i, s in enumerate(pl):
        if i!=0 and s.startswith('im'):
            pl[i] = s.replace('im', '')
    crop_flag = True
    for i, s in enumerate(pl):
        if s != 's' and s != 'j':
            continue
        start, end = int(pl[i+1])-1, int(pl[i+2])
        for j in range(start, end):
            img = Image.open(os.path.join(image_path, f'{patient}_{j:04d}.png'))
            img = np.array(img)
            try:
                x1, y1, x2, y2 = find_coordinate(*img.shape[0:2], os.path.join(label_path, f'{patient}_{j:04d}.txt'), s)
            except:
                traceback.print_exc()
                crop_flag = False
            else:#if crop_flag:
                crop = img[y1:y2, x1:x2]
                crop = Image.fromarray(crop)
                crop.save(os.path.join(crop_path, f'{patient}_{s}_{j:04d}.png'))
    if not crop_flag:
        #print('Delete crop_path')
        #shutil.rmtree(crop_path)
        error_patient_list.append(patient)

******Processing baozuoshi-J-16-79******
******Processing changzhenxuan-J-15-55******
******Processing chenaijiao-J-26-41******
******Processing chencaiying-J-19-79******
******Processing chenfuliang-S-21-31-J-21-84******


Traceback (most recent call last):
  File "<ipython-input-101-5f34f1b3dad3>", line 47, in <module>
    x1, y1, x2, y2 = find_coordinate(*img.shape[0:2], os.path.join(label_path, f'{patient}_{j:04d}.txt'), s)
  File "<ipython-input-101-5f34f1b3dad3>", line 15, in find_coordinate
    assert 0.25 < corr1[1] < 0.75 and 0.25 < corr1[2] < 0.75, f'边界框1中心({corr1[1]}, {corr1[2]})出界：{label_file.split("/")[-1]}'
AssertionError: 边界框1中心(0.791016, 0.591797)出界：chenfuliang-S-21-31-J-21-84_0058.txt
Traceback (most recent call last):
  File "<ipython-input-101-5f34f1b3dad3>", line 47, in <module>
    x1, y1, x2, y2 = find_coordinate(*img.shape[0:2], os.path.join(label_path, f'{patient}_{j:04d}.txt'), s)
  File "<ipython-input-101-5f34f1b3dad3>", line 15, in find_coordinate
    assert 0.25 < corr1[1] < 0.75 and 0.25 < corr1[2] < 0.75, f'边界框1中心({corr1[1]}, {corr1[2]})出界：{label_file.split("/")[-1]}'
AssertionError: 边界框1中心(0.790039, 0.591797)出界：chenfuliang-S-21-31-J-21-84_0059.txt


******Processing chenganyao-J-19-40******
******Processing chenhui-S-17-25-J-17-87******
******Processing chenjufa-J-16-66******
******Processing digenlan-J-23-50******


Traceback (most recent call last):
  File "<ipython-input-101-5f34f1b3dad3>", line 47, in <module>
    x1, y1, x2, y2 = find_coordinate(*img.shape[0:2], os.path.join(label_path, f'{patient}_{j:04d}.txt'), s)
  File "<ipython-input-101-5f34f1b3dad3>", line 5, in find_coordinate
    with open(label_file, 'r') as f:
FileNotFoundError: [Errno 2] No such file or directory: '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/new/digenlan-J-23-50/labels/digenlan-J-23-50_0022.txt'
Traceback (most recent call last):
  File "<ipython-input-101-5f34f1b3dad3>", line 47, in <module>
    x1, y1, x2, y2 = find_coordinate(*img.shape[0:2], os.path.join(label_path, f'{patient}_{j:04d}.txt'), s)
  File "<ipython-input-101-5f34f1b3dad3>", line 5, in find_coordinate
    with open(label_file, 'r') as f:
FileNotFoundError: [Errno 2] No such file or directory: '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/new/digenlan-J-23-50/labels/digenlan-J-23-50_0025.txt'
Traceback (most recent call last):
  File "<ipython-input-101-5f

******Processing fangchunfeng-J-16-48******
******Processing fangzhuxiang-J-18-61******
******Processing fudehe-S-20-33-J-20-70******
******Processing ganxiaobin-J-23-51******


...

Traceback (most recent call last):
  File "<ipython-input-101-5f34f1b3dad3>", line 47, in <module>
    x1, y1, x2, y2 = find_coordinate(*img.shape[0:2], os.path.join(label_path, f'{patient}_{j:04d}.txt'), s)
  File "<ipython-input-101-5f34f1b3dad3>", line 9, in find_coordinate
    assert aorta == 'j', f'如果只有一个label那么此时应为降主动脉, 但实际为{aorta}：{label_file.split("/")[-1]}'
AssertionError: 如果只有一个label那么此时应为降主动脉, 但实际为s：zhengchong-S-16-31-J-16-87_0018.txt


******Processing zhongchuankun-J-17-69******


Traceback (most recent call last):
  File "<ipython-input-101-5f34f1b3dad3>", line 47, in <module>
    x1, y1, x2, y2 = find_coordinate(*img.shape[0:2], os.path.join(label_path, f'{patient}_{j:04d}.txt'), s)
  File "<ipython-input-101-5f34f1b3dad3>", line 5, in find_coordinate
    with open(label_file, 'r') as f:
FileNotFoundError: [Errno 2] No such file or directory: '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/new/zhongchuankun-J-17-69/labels/zhongchuankun-J-17-69_0041.txt'
Traceback (most recent call last):
  File "<ipython-input-101-5f34f1b3dad3>", line 47, in <module>
    x1, y1, x2, y2 = find_coordinate(*img.shape[0:2], os.path.join(label_path, f'{patient}_{j:04d}.txt'), s)
  File "<ipython-input-101-5f34f1b3dad3>", line 5, in find_coordinate
    with open(label_file, 'r') as f:
FileNotFoundError: [Errno 2] No such file or directory: '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/new/zhongchuankun-J-17-69/labels/zhongchuankun-J-17-69_0061.txt'


******Processing zhoujianglong-S-17-31******
******Processing zhuxinjian-J-20-23******


Traceback (most recent call last):
  File "<ipython-input-101-5f34f1b3dad3>", line 47, in <module>
    x1, y1, x2, y2 = find_coordinate(*img.shape[0:2], os.path.join(label_path, f'{patient}_{j:04d}.txt'), s)
  File "<ipython-input-101-5f34f1b3dad3>", line 5, in find_coordinate
    with open(label_file, 'r') as f:
FileNotFoundError: [Errno 2] No such file or directory: '/nfs3-p1/zsxm/dataset/aorta_ct_img_label/new/zhuxinjian-J-20-23/labels/zhuxinjian-J-20-23_0019.txt'


In [102]:
print(len(error_patient_list))
print(error_patient_list)

13
['chenfuliang-S-21-31-J-21-84', 'digenlan-J-23-50', 'ganxiaobin-J-23-51', 'kongxiaoqing-S-19-32-J-27-94', 'liangjianjun-S-17-23', 'liuhongfu-J-57-200', 'pancunxiao-S-16-30-J-16-70', 'siyoulin-S-16-26', 'wangxiaofu-S-19-29-J-19-65', 'yuguiying-S-16-22-J-16-77', 'zhengchong-S-16-31-J-16-87', 'zhongchuankun-J-17-69', 'zhuxinjian-J-20-23']
